# Fine tuning Marian-NMT en-ru model

## Установка зависимостей

In [ ]:
!pip install datasets transformers[sentencepiece]
!pip install sacrebleu
!pip install accelerate
!pip install openpyxl
!apt install git-lfs
!pip install matplotlib

In [ ]:
# загрузим репозиторий; нужен для предобработки
!git clone https://github.com/eleldar/Translator.git

In [117]:
# загрузка исходной модели
!git clone https://huggingface.co/Helsinki-NLP/opus-mt-en-ru && ls opus-mt-en-ru

Cloning into 'opus-mt-en-ru'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 58 (delta 30), reused 58 (delta 30), pack-reused 0
Unpacking objects: 100% (58/58), done.
Filtering content: 100% (2/2), 829.76 MiB | 20.66 MiB/s, done.
README.md    pytorch_model.bin	source.spm  tokenizer_config.json
config.json  rust_model.ot	target.spm  vocab.json


## Настройка git

In [1]:
!git config --global user.email "eleldar@mail.ru"
!git config --global user.name "eleldar"
!git config --global credential.helper store

## Импортирование зависимостей

In [2]:
import os
import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from time import gmtime, strftime
from huggingface_hub import Repository
from accelerate import Accelerator, notebook_launcher
import datasets
from datasets import (
    Dataset, DatasetDict, load_dataset, load_metric,
    concatenate_datasets, interleave_datasets
)
import transformers
from transformers import (
    AdamW, AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,
    get_constant_schedule, get_constant_schedule_with_warmup, get_cosine_schedule_with_warmup, 
    get_linear_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup, 
    get_polynomial_decay_schedule_with_warmup
)

## Загрузка данных

### Общий корпус длинных предложений

In [3]:
normal_url = 'https://github.com/eleldar/Translator/blob/master/test_dataset/flores101_dataset/101_languages.xlsx?raw=true'
normal_df = pd.read_excel(normal_url)[["eng", "rus"]].rename(columns={"eng": "en", "rus": "ru"})
normal_df

,en,ru
0,"On Monday, scientists from the Stanford Univer...",В понедельник ученые из Медицинской школы Стэн...
1,Lead researchers say this may bring early dete...,"Ведущие исследователи утверждают, что он может..."
2,The JAS 39C Gripen crashed onto a runway at ar...,Приблизительно в 9:30 по местному времени (02:...
3,The pilot was identified as Squadron Leader Di...,Личность пилота была установлена. Им оказался ...
4,Local media reports an airport fire vehicle ro...,"Местные СМИ сообщают, что в аэропорту по пути ..."
...,...,...
2004,"As the areas are sparsely populated, and light...","Так как эти районы являются малонаселенными, и..."
2005,Japanese work culture is more hierarchical and...,Японская культура труда иерархичнее и формальн...
2006,"Suits are standard business attire, and cowork...","Костюмы являются стандартной деловой одеждой, ..."
2007,"Workplace harmony is crucial, emphasizing grou...","Чрезвычайно важна гармония на рабочем месте, а..."


### Общий корпус кратких предложений

In [4]:
short_url = 'https://github.com/eleldar/Translator/blob/master/test_dataset/normal.xlsx?raw=true'
short_df = pd.read_excel(short_url).rename(columns={"en_sent": "en", "ru_sent": "ru"})
short_df

,en,ru
0,Good morning!,Доброе утро!
1,Did the alarm clock go off?,Будильник звонил?
2,It's time to get up.,Время вставать!
3,Get up soon.,Вставай быстрее.
4,Are you awake?,Ты проснулся?
...,...,...
2659,Never say die.,Никогда не сдавайся.
2660,Let bygones be bygones.,Пусть прошедшее будет прошедшим.
2661,Better than nothing.,"Лучше, чем ничего."
2662,A fair-weather friend.,Ненадёжный друг.


### Предметный корпус

In [5]:
subject_url = 'https://github.com/eleldar/Translator/blob/master/test_dataset/corrected_vocab.xlsx?raw=true'
subject_df = pd.read_excel(subject_url).drop(columns=['en_keys', 'ru_keys']).rename(columns={"en_sent": "en", "ru_sent": "ru"})
subject_df

,en,ru
0,Please contact us as soon as possible,"пожалуйста, свяжитесь с нами как можно скорее."
1,If you would like to have an additional inform...,если вы хотите получить дополнительную информа...
2,This task is difficult for me.,Это задание трудное для меня.
3,Our task is to increase channel activity in th...,"Наша задача состоит в том, чтобы увеличить акт..."
4,My employee never completes tasks on time.,Мой сотрудник никогда не делает задания в срок.
...,...,...
1033,Disable anti-virus firewall and open up port 443.,Отключите брандмауэр для защиты от вирусов и о...
1034,We need to find a way to break through that si...,Нам нужно найти способ обойти брандмауэр сайта.
1035,Several representatives supported incremental ...,Ряд представителей высказались за поэтапное ра...
1036,Incremental development increases affordabilit...,Поэтапное развитие повышает доступность за сче...


### Тестовый корпус (из модели)

In [6]:
test_url = 'https://github.com/eleldar/Translator/blob/master/test_dataset/test_opus_en-ru_dataset.xlsx?raw=true'
test_df = pd.read_excel(test_url).drop(columns=['Unnamed: 0'])
test_df

,en,ru
0,Have you ever been to Switzerland?,Ты уже бывал в Швейцарии?
1,She learned quickly.,Она быстро училась.
2,No one tried to stop me.,Никто не пытался меня остановить.
3,Guilds were an important part of society in th...,Гильдии были важной частью общества в Средние ...
4,You say that it is your custom to burn widows....,"Вы говорите, что сжигать вдов - ваш обычай. Оч..."
...,...,...
4995,You may sit anywhere you like.,Ты можешь сесть где хочешь.
4996,There's a lot I want to talk to you about.,Я о многом хочу с вами поговорить.
4997,Let me tell Tom.,Позволь мне рассказать Тому.
4998,"At twilight, snow looks blue.",В сумерках снег кажется синим.


## Предобработка данных

> Требуется замена символов юникода, т.к. встроенный токенизатор этого не выполняет

In [7]:
os.getcwd()

'/mnt/home'

In [8]:
# переключим на каталог с импортируемыми модулями
os.chdir('/mnt/home/Translator/OpenAPI/')

In [9]:
os.getcwd()

'/mnt/home/Translator/OpenAPI'

In [10]:
!ls

README.md    api		   main.py  requirements.txt
__pycache__  documented_endpoints  models   venv


In [11]:
# импортировали
from api.tools.preprocess import get_commands, preprocess_text

In [12]:
# словарь команд для предобработки на основе файла с расширением направления перевода и checkpoints
checkpoints = {'en-ru', 'ar-ru', 'ru-ar', 'ru-en', 'en-ar', 'ar-en'}
commands = get_commands(checkpoints)
list(commands['en-ru'])[:5], list(commands['ru-en'])[:5]

([('～', '\\~'), ('〉', '\\>'), ('？', '\\?'), ('；', ';'), ('。 *', '. ')],
 [('～', '\\~'), ('〉', '\\>'), ('？', '\\?'), ('；', ';'), ('。 *', '. ')])

In [13]:
# замена спецсимволов
# normalisation = lambda text: 1 # preprocess_text(commands['en-ru'], text['en_sent']) if direct in commands else text['en_sent']
def normalisation(text):
    text['en'] = preprocess_text(commands['en-ru'], text['en'])
    text['ru'] = preprocess_text(commands['ru-en'], text['ru'])
    return text

In [14]:
# вернули рабочую директорию
os.chdir('/mnt/home')

## Сборка наборов данных

### Создадим объекты Dataset

In [15]:
# Общий корпус длинных предложений
# normal_df
normal_dataset = Dataset.from_pandas(normal_df)
normal_dataset = normal_dataset.map(normalisation)
normal_dataset

  0%|          | 0/2009 [00:00<?, ?ex/s]

Dataset({
    features: ['en', 'ru'],
    num_rows: 2009
})

In [16]:
# Общий корпус кратких предложений
# short_df
short_dataset = Dataset.from_pandas(short_df)
short_dataset = short_dataset.map(normalisation)
short_dataset

  0%|          | 0/2664 [00:00<?, ?ex/s]

Dataset({
    features: ['en', 'ru'],
    num_rows: 2664
})

In [17]:
# Предметный корпус
# subject_df
subject_dataset = Dataset.from_pandas(subject_df).shuffle()
subject_dataset = subject_dataset.map(normalisation)
subject_dataset

  0%|          | 0/1038 [00:00<?, ?ex/s]

Dataset({
    features: ['en', 'ru'],
    num_rows: 1038
})

In [18]:
# Тестовый корпус
# test_df
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(normalisation)
test_dataset

  0%|          | 0/5000 [00:00<?, ?ex/s]

Dataset({
    features: ['en', 'ru'],
    num_rows: 5000
})

### Объединим обучающую часть предметного и тестовые набора

In [19]:
# целевой "словарь"
split_datasets = DatasetDict()

In [20]:
split_datasets['normal'] = normal_dataset
split_datasets['short'] = short_dataset
split_datasets

DatasetDict({
    normal: Dataset({
        features: ['en', 'ru'],
        num_rows: 2009
    })
    short: Dataset({
        features: ['en', 'ru'],
        num_rows: 2664
    })
})

In [21]:
sub_train_and_test = subject_dataset.train_test_split(test_size=0.2)
sub_train_and_test

DatasetDict({
    train: Dataset({
        features: ['en', 'ru'],
        num_rows: 830
    })
    test: Dataset({
        features: ['en', 'ru'],
        num_rows: 208
    })
})

In [22]:
tmp = test_dataset.train_test_split(test_size=0.166)
tmp

DatasetDict({
    train: Dataset({
        features: ['en', 'ru'],
        num_rows: 4170
    })
    test: Dataset({
        features: ['en', 'ru'],
        num_rows: 830
    })
})

In [23]:
split_datasets['train'] = interleave_datasets(
    [sub_train_and_test['train'], tmp['test']]
).shuffle()
split_datasets['validation'] = sub_train_and_test.pop("test")
split_datasets

DatasetDict({
    normal: Dataset({
        features: ['en', 'ru'],
        num_rows: 2009
    })
    short: Dataset({
        features: ['en', 'ru'],
        num_rows: 2664
    })
    train: Dataset({
        features: ['en', 'ru'],
        num_rows: 1660
    })
    validation: Dataset({
        features: ['en', 'ru'],
        num_rows: 208
    })
})

In [24]:
## Расскоментровать для использования всего модельного датасета для обучения; также в функции приедется изменить методы оценки
# split_datasets['train'] = concatenate_datasets(
#     [sub_train_and_test['train'], test_dataset]
# ).shuffle()
# split_datasets['validation'] = sub_train_and_test.pop("test")
# split_datasets

In [25]:
split_datasets['test'] = tmp['train']
split_datasets

DatasetDict({
    normal: Dataset({
        features: ['en', 'ru'],
        num_rows: 2009
    })
    short: Dataset({
        features: ['en', 'ru'],
        num_rows: 2664
    })
    train: Dataset({
        features: ['en', 'ru'],
        num_rows: 1660
    })
    validation: Dataset({
        features: ['en', 'ru'],
        num_rows: 208
    })
    test: Dataset({
        features: ['en', 'ru'],
        num_rows: 4170
    })
})

## Функция обучения

> Не поддерживает использование в качестве метода либо в цикл (проверено эмпирическим путем), т.к. используется параллельное использование нескольких GPU

In [ ]:
lr_schedulers = ['get_constant_schedule', 'get_constant_schedule_with_warmup',
                 'get_cosine_schedule_with_warmup', 'get_cosine_with_hard_restarts_schedule_with_warmup',
                 'get_linear_schedule_with_warmup', 'get_polynomial_decay_schedule_with_warmup',
                 'torch_optim_lr_scheduler_one_cycle_lr']

hyperparameters = {
    "learning_rate": 1e-6,
    "num_epochs": 2,
    "train_batch_size": 8,
    "eval_batch_size": 32, 
    "model_checkpoint": "opus-mt-en-ru",
    "max_input_length": 128,
    "max_target_length": 128,
    "max_generate_length": 128,    
    "output_dir": f'experiences/fine_tuned_en_ru_model_{strftime("%Y-%m-%d_%H-%M-%S", gmtime())}',
    "file_scores": 'scores.txt',
    "scheduler": lr_schedulers[0], # настраиваемый параметр
}

tokenizer = AutoTokenizer.from_pretrained(hyperparameters["model_checkpoint"], return_tensors="pt")

def preprocess_function(examples, hyperparameters=hyperparameters, tokenizer=tokenizer):
    '''Получение IDs'''
    model_inputs = tokenizer(examples["en"], max_length=hyperparameters['max_input_length'], truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["ru"], max_length=hyperparameters['max_target_length'], truncation=True)
    model_inputs["labels"] = labels["input_ids"] # присвоили исходному языку IDs целевого языка
    return model_inputs


def postprocess(predictions, labels, tokenizer=tokenizer):
    '''Получение текста из IDs'''
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()
    # Декодированные токены из IDs, спрогнозированные моделью
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Замена -100 в метках, так как их нельзя декодировать.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Декодированные метки токены из IDs, являющиеся эталонным переводом
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Пост-обрабработка, т.к. для прогноза нужен список, а для эталона список списков
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels


def evaluate(model, accelerator, examples, epoch='base', note="sub", hyperparameters=hyperparameters):
    '''Оценка'''
    metric = load_metric("sacrebleu")
    model.eval()
    for batch in tqdm(examples):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=hyperparameters["max_generate_length"],
            )
        labels = batch["labels"]
        # Необходимое выравнивание для заполнения прогнозов и меток для метода accelerator.gather()
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)
        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)
        # подготовка данных для оценки
        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        # примечение пакетной метрики
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)
    results = metric.compute()
    response = f"{note}_score for {epoch} epoch: {results['score']}\n"
    with open(f"{hyperparameters['output_dir']}_{hyperparameters['scheduler']}/{hyperparameters['file_scores']}", 'a') as file:
        file.write(response)
    print(f"{note}_score for epoch {epoch}, BLEU score: {results['score']:.2f}")
    

def get_image(hyperparameters):
    '''Создание и сохранение графика'''
    with open(f"{hyperparameters['output_dir']}_{hyperparameters['scheduler']}/{hyperparameters['file_scores']}") as f:
        score = f.readlines()
    sub = [float(i.strip().split(': ')[1]) for i in score[0::4][0::4]]
    normal = [float(i.strip().split(': ')[1]) for i in score[0::4][1::4]]
    short = [float(i.strip().split(': ')[1]) for i in score[0::4][2::4]]
    test = [float(i.strip().split(': ')[1]) for i in score[0::4][3::4]]
    X = [i for i in range(hyperparameters["num_epochs"] + 1)]
    Y = [i for i in range(0, 61)]
    score_df = pd.DataFrame({'Предметный': sub, 'Обычные': normal, 'Короткие': short, 'Модельные': test})
    mx_sub = max(sub)
    inx = sub.index(mx_sub)
    modscore = test[inx]
    img = score_df.plot(xticks=X, yticks=Y, style='^', figsize=(15,12));
    img.axvline(inx, color='grey')
    img.legend(loc='lower left')
    img.set_xlabel("Epochs")
    img.set_ylabel("BLEU")
    img.annotate(f'sub {mx_sub:.2f}', xy=(inx, mx_sub), xytext=(inx, mx_sub),
            arrowprops=dict(facecolor='blue', shrink=0.05),
            )
    img.annotate(f'mod {modscore:.2f}', xy=(inx, modscore), xytext=(inx, modscore),
            arrowprops=dict(facecolor='red', shrink=0.05),
            )
    img.annotate(f"{hyperparameters['scheduler'].upper()} by LR:{hyperparameters['learning_rate']}", xy=(0, 58), xytext=(0, 58))
    directory = f"{hyperparameters['output_dir']}_{hyperparameters['scheduler']}"
    img.get_figure().savefig(f"{directory}/maxsub-{mx_sub:.2f}_mod-{modscore:.2f}_epoch-{inx}_{hyperparameters['scheduler']}.png")    
    
def training_function(hyperparameters, tokenized_datasets, tokenizer):
    directory = f'{hyperparameters["output_dir"]}_{hyperparameters["scheduler"]}'
    try:
        repo = Repository(directory, clone_from='eleldar/train')
    except Exception as e:
        pass
    if not os.path.isfile(f"{hyperparameters['output_dir']}/{hyperparameters['file_scores']}_{hyperparameters['scheduler']}"):
        with open(f"{hyperparameters['output_dir']}_{hyperparameters['scheduler']}/{hyperparameters['file_scores']}", 'w') as file: # файл для складывания оценок
            file.write('')
        with open(f"{hyperparameters['output_dir']}_{hyperparameters['scheduler']}/.gitignore", 'w') as file:
            file.write("*.png\n")
    accelerator = Accelerator()
    if accelerator.is_main_process:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()
    
    model = AutoModelForSeq2SeqLM.from_pretrained(hyperparameters["model_checkpoint"])
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    
    tokenized_datasets.set_format("torch")
    train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, 
                                  collate_fn=data_collator, batch_size=hyperparameters['train_batch_size'])
    eval_dataloader = DataLoader(tokenized_datasets["validation"], shuffle=False,
                                 collate_fn=data_collator, batch_size=hyperparameters['eval_batch_size'])
    normal_dataloader = DataLoader(tokenized_datasets["normal"], shuffle=False,
                                 collate_fn=data_collator, batch_size=hyperparameters['eval_batch_size'])
    short_dataloader = DataLoader(tokenized_datasets["short"], shuffle=False,
                                 collate_fn=data_collator, batch_size=hyperparameters['eval_batch_size'])
    test_dataloader = DataLoader(tokenized_datasets["test"], shuffle=False,
                                 collate_fn=data_collator, batch_size=hyperparameters['eval_batch_size'])

    optimizer = AdamW(model.parameters(), lr=hyperparameters["learning_rate"])    
    model, optimizer, train_dataloader, eval_dataloader, normal_dataloader, short_dataloader, test_dataloader = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader, normal_dataloader, short_dataloader, test_dataloader
    )
    num_epochs = hyperparameters["num_epochs"]
    lr_schedulers = {'get_constant_schedule': get_constant_schedule(
            optimizer=optimizer
        ),
            'get_constant_schedule_with_warmup': get_constant_schedule_with_warmup(
            optimizer=optimizer, num_warmup_steps=100
        ),
        'get_cosine_schedule_with_warmup': get_cosine_schedule_with_warmup(
            optimizer=optimizer, num_warmup_steps=100, 
            num_training_steps=len(train_dataloader) * num_epochs,
            num_cycles=0.5
        ),
        'get_cosine_with_hard_restarts_schedule_with_warmup': get_cosine_with_hard_restarts_schedule_with_warmup(
            optimizer=optimizer, num_warmup_steps=100,
            num_training_steps=len(train_dataloader) * num_epochs,
            num_cycles=1
        ),
        'get_linear_schedule_with_warmup': get_linear_schedule_with_warmup(
            optimizer=optimizer, num_warmup_steps=100,
            num_training_steps=len(train_dataloader) * num_epochs,
        ),
        'get_polynomial_decay_schedule_with_warmup': get_polynomial_decay_schedule_with_warmup(
            optimizer=optimizer, num_warmup_steps=100,
            num_training_steps=len(train_dataloader) * num_epochs,
            lr_end=1e-7, power=1.0
        ),
        'torch_optim_lr_scheduler_one_cycle_lr': torch.optim.lr_scheduler.OneCycleLR(
            optimizer=optimizer, max_lr=1e-5, pct_start=1 / (num_epochs),
            total_steps=len(train_dataloader) * num_epochs + 10, div_factor=1e+3, final_div_factor=1e+4,
            anneal_strategy='cos'
        )
    }
    lr_scheduler = lr_schedulers[hyperparameters['scheduler']]

    # оценка перед обучением
    evaluate(model, accelerator, eval_dataloader, note="sub")
    evaluate(model, accelerator, normal_dataloader, note="normal")
    evaluate(model, accelerator, short_dataloader, note="short")
    evaluate(model, accelerator, test_dataloader, note="test")
    
    try:
        repo.git_add(".")
        repo.git_commit(commit_message="base and gitignore")
    except Exception as e:
        pass

    # обучение    
    progress_bar = tqdm(range(num_epochs * len(train_dataloader)), disable=not accelerator.is_main_process)
    for epoch in range(1, num_epochs + 1):
        model.train()
        for batch in train_dataloader:
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
        
        # оценка в момент обучения
        evaluate(model, accelerator, eval_dataloader, epoch=epoch, note="sub")
        evaluate(model, accelerator, normal_dataloader, epoch=epoch, note="normal")
        evaluate(model, accelerator, short_dataloader, epoch=epoch, note="short")
        evaluate(model, accelerator, test_dataloader, epoch=epoch, note="test")        
        
        # Сохранение и обновление
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(directory, save_function=accelerator.save)
        if accelerator.is_main_process:
            tokenizer.save_pretrained(directory)
            try:                
                repo.git_add(".")                
                repo.git_commit(commit_message=f"Training in progress epoch {epoch}")
            except Exception as e:
                pass
    get_image(hyperparameters)
    
def decorator(function, *args):
    '''для добавление аргументов в функцию для обучения'''
    def wrapper():
        return function(*args)
    return wrapper

tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)    
training_function = decorator(training_function, hyperparameters, 
                              tokenized_datasets, tokenizer
                             )
notebook_launcher(training_function, num_processes=4)

# Использование

In [35]:
split_datasets['validation'][60]

{'en': 'Companies need to buy routers to direct data traffic and connect to the internet.',
 'ru': 'Компаниям необходимо покупать роутеры для направления трафика данных и подключения к Интернету.'}

In [36]:
# до
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-ru"
translator = pipeline("translation", model=model_checkpoint)
translator("Companies need to buy routers to direct data traffic and connect to the internet.")

[{'translation_text': 'Компании должны покупать маршрутизаторы для управления трафиком данных и подключения к Интернету.'}]

In [37]:
# после
from transformers import pipeline

model_checkpoint = hyperparameters['output_dir']
translator = pipeline("translation", model=model_checkpoint)
translator("Companies need to buy routers to direct data traffic and connect to the internet.")

[{'translation_text': 'Компании должны покупать роутеры для управления трафиком данных и подключения к интернету.'}]